In [3]:
import time
import datetime
import requests
from IPython.display import clear_output

## [START HERE] Enter your input here 

In [4]:
binance = {
    'BTCUSDT': 0.0,
    'ETHUSDT': 0.0,
    'ADAUSDT': 0.0,
    'XRPUSDT': 131.791,
    'USDT': 904.7
}

cold_wallet = {
    'BTCUSDT': 0.0756004,
    'ETHUSDT': 4.8333,
    'ADAUSDT': 6482.38
}

avg_cost = {
    'BTCUSDT': 91219.32,
    'ETHUSDT': 2873.30,
    'ADAUSDT': 0.8472,
    'XRPUSDT': 2.54
}

## Specific Search

In [5]:
item_to_search = 'ETHBTC'
url_root = "https://api.binance.com/api/v3/ticker/price?symbol="

response = requests.get(url_root+item_to_search)
if response.status_code == 200:
    price = response.json()
    print(f"The current price of {price['symbol']} is: {price['price']}")
else:
    print(f"Error：{response.status_code} - {response.text}")

The current price of ETHBTC is: 0.02328000


## Asset & Distribution Calculation

In [6]:
total_owned = {}
for key, value in binance.items():
    total_owned[key] = value if key not in cold_wallet.keys() else round(value + cold_wallet[key], 5)

portion = { key: round((total_owned[key] * avg_cost[key]), 5) if key in avg_cost else total_owned[key] for key in total_owned.keys() }
total_cost = sum(portion.values())
percentage = { key: round((value / total_cost * 100), 2) for key, value in portion.items() }
print("Your current owned assets:")
for key, value in total_owned.items():
    print(f'{key if key == 'USDT' else key[:-4]}: {value} - with avg cost {[avg_cost[key]] if key in avg_cost.keys() else '[Not Avaliable]'}')
print("\nYour current asset's distribution is:")
for key, value in percentage.items():
    if key != 'USDT' and key != 'USDC':
        print(f"{key[:-4]}: {value}%")
    else:
        print(f"{key}: {value}%")
print(f"\nTotal cost so far: {total_cost}")

Your current owned assets:
BTC: 0.0756 - with avg cost [91219.32]
ETH: 4.8333 - with avg cost [2873.3]
ADA: 6482.38 - with avg cost [0.8472]
XRP: 131.791 - with avg cost [2.54]
USDT: 904.7 - with avg cost [Not Avaliable]

Your current asset's distribution is:
BTC: 25.06%
ETH: 50.47%
ADA: 19.96%
XRP: 1.22%
USDT: 3.29%

Total cost so far: 27515.02296


### Real-time Unrealized Profits.

In [10]:
filter_symbols = [key for key in avg_cost.keys()]
url_custom = [ url_root + coin for coin in filter_symbols]
start_prices = {}

for url in url_custom:
    response = requests.get(url)
    if response.status_code == 200:
        price = response.json()
        start_prices[price['symbol']] = float(price['price']) 
    else:
        print(f"Error：{response.status_code} - {response.text}")

start_prices

{'BTCUSDT': 85733.12, 'ETHUSDT': 2022.78, 'ADAUSDT': 0.729, 'XRPUSDT': 2.4873}

In [18]:
fetch_frequence = 10
current_prices = {}
default_color = '\033[0m'

while True:
    clear_output(wait=True)
    current_time = datetime.datetime.now()
    for url in url_custom:
        response = requests.get(url)
        if response.status_code == 200:
            price = response.json()
            current_prices[price['symbol']] = float(price['price'])
        else:
            print(f"Error：{response.status_code} - {response.text}")
            
    print('[' + current_time.strftime('%Y/%m/%d %H:%M:%S')+ ']')
    print("The current price of your owned coins are:\n")
    print('{:^8}'.format('Type') + ' ' + '{:^10}'.format('Price') + ' ' + '{:^7}'.format('I/D') + '   ' + '{:^7}'.format('Per'))
    for key, value in current_prices.items():
        diff = round(value - start_prices[key], 4)
        percentage = round(diff / start_prices[key] * 100, 4)
        char = '+' if diff > 0 else '-'
        color = default_color if diff == 0 else ('\033[92m' if diff > 0 else '\033[91m')
        diff, percentage = abs(diff), abs(percentage)
        print('%-8s' % key + ':', end = ' ')
        print('%-10s' % round(value, 5), end='')
        print(f"{color} {char}", end = '')
        print('%-7s' % diff, end = ' ')
        print(f"{default_color} ({color}{char}", end = '')
        print('%-7s' % percentage + f'%{default_color})')
    
    unrealized_profit = { key: round((float(current_prices[key]) - avg_cost[key])* total_owned[key], 5) for key, value in avg_cost.items() }
    print("\nThe unrealized profits of your owned coins are:")
    total_unrealized_profit = 0
    for key, value in unrealized_profit.items():
        total_unrealized_profit += value
        c = '+' if value > 0 else '-'
        color = default_color if value == 0 else ('\033[92m' if value > 0 else '\033[91m')
        print(f'%-8s' % key + ': ' + f'{color}{c}' + f"{'%-12s' % abs(round(value, 5))}", end = '')
        print(default_color)

    color = default_color if value == 0 else ('\033[92m' if value > 0 else '\033[91m')
    print(f"The total unrealized profits is: {color}{round(total_unrealized_profit, 5)}")
    update = False
    time.sleep(fetch_frequence)

[2025/03/26 09:21:22]
The current price of your owned coins are:

  Type     Price      I/D       Per  
BTCUSDT : 87851.37   +2118.25  (+2.4707 %)
ETHUSDT : 2074.76    +51.98    (+2.5697 %)
ADAUSDT : 0.7546     +0.0256   (+3.5117 %)
XRPUSDT : 2.4765     -0.0108   (-0.4342 %)

The unrealized profits of your owned coins are:
BTCUSDT : -254.61702   
ETHUSDT : -3859.58338  
ADAUSDT : -600.26839   
XRPUSDT : -8.36873     
The total unrealized profits is: -4722.83752


KeyboardInterrupt: 